# Competencia 01

## Carga de librerias y datos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle
import gc

In [ ]:
ganancia_acierto = 780000
costo_estimulo = 20000

semillas = [600011, 600043, 600053, 600071, 600073]

In [ ]:
base_path =  'path/'
dataset_path = base_path + 'datos/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe_1.csv'

In [ ]:
chunk_size = 50000
meses_a_filtrar = [202106, 202105, 202104, 202103, 202102, 202101]
nombres_df = ['df_junio', 'df_mayo', 'df_abril', 'df_marzo', 'df_febrero', 'df_enero']
dataframes_por_mes = {}

for mes, nombre_df in zip(meses_a_filtrar, nombres_df):
    chunks = pd.read_csv(dataset_path + dataset_file, chunksize=chunk_size)
    lista_chunks_filtrados = []

    for i, chunk in enumerate(chunks):
        chunk_filtrado = chunk[chunk['foto_mes'] == mes]
        lista_chunks_filtrados.append(chunk_filtrado)

    dataframes_por_mes[nombre_df] = pd.concat(lista_chunks_filtrados, ignore_index=True)

df_junio = dataframes_por_mes['df_junio']
df_mayo = dataframes_por_mes['df_mayo']
df_abril = dataframes_por_mes['df_abril']
df_marzo = dataframes_por_mes['df_marzo']
df_febrero = dataframes_por_mes['df_febrero']
df_enero = dataframes_por_mes['df_enero']

In [ ]:
dataframes = [df_junio, df_mayo, df_abril, df_marzo, df_febrero, df_enero]

for df in dataframes:
    df['clase_peso'] = 1.0
    df.loc[df['clase_ternaria'] == 'baja+2', 'clase_peso'] = 1.00003
    df.loc[df['clase_ternaria'] == 'baja+1', 'clase_peso'] = 1.00001

for df in dataframes:
    df['clase_binaria2'] = 0
    df['clase_binaria2'] = np.where(df['clase_ternaria'] == 'continua', 0, 1)
    df['clase_binaria1'] = 0
    df['clase_binaria1'] = np.where(df['clase_ternaria'] == 'baja+2', 1, 0)

## Optimización de hiperparámetros

Optimizo los hiperparámetros con los continúa de marzo + los baja+1 y baja +2 de enero, febrero y marzo. Se testea con Abril.

In [ ]:
dfs = [df_enero, df_febrero, df_marzo, df_abril, df_mayo, df_junio]

clientes_baja = set()
for dfx in dfs:
    clientes_baja.update(
        dfx.loc[dfx['clase_ternaria'].isin(['baja+1', 'baja+2']), 'numero_de_cliente'].to_numpy()
    )
gc.collect()

train_data_parts = []
meses_interes = {202101, 202102, 202103} 

for dfx in dfs:
    mask = (
        ~((dfx['numero_de_cliente'].isin(clientes_baja)) &
          (dfx['clase_ternaria'] == 'continua')) &
        (dfx['foto_mes'].isin(meses_interes))
    )
    train_data_parts.append(dfx.loc[mask])
    gc.collect()

_train_data = pd.concat(train_data_parts, ignore_index=True)
train_data = _train_data[~(_train_data['foto_mes'].isin([202101, 202102]) & (_train_data['clase_ternaria'] == 'continua'))]

del dfs, dfx, clientes_baja, train_data_parts, mask, _train_data
gc.collect()

In [ ]:
X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria1','clase_binaria2','clase_resumida','foto_mes'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
y_train_binaria1 = train_data['clase_binaria1']
w_train = train_data['clase_peso']

test_data = df_abril
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria1','clase_binaria2','clase_resumida','foto_mes'], axis=1)
y_test_binaria2 = test_data['clase_binaria2']
y_test_binaria1 = test_data['clase_binaria1']
y_test_class = test_data['clase_ternaria']
w_test = test_data['clase_peso']

In [ ]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00003, ganancia_acierto, 0) - np.where(weight < 1.00003, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

In [ ]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 5, 40) 
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 200, 5000) 
    feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.6) 
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.5, 1.0) 
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2)
    reg_alpha = trial.suggest_float('reg_alpha', 0.5, 50.0, log=True) 
    reg_lambda = trial.suggest_float('reg_lambda', 0.5, 50.0, log=True) 
    max_depth = trial.suggest_int('max_depth', 3, 20) 
    
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'max_bin': 31,
        'num_leaves': num_leaves,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'learning_rate': learning_rate,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda,
        'max_depth': max_depth,
        'seed': semillas[0],
        'verbose': -1
    }

    train_data = lgb.Dataset(X_train, label=y_train_binaria2, weight=w_train)

    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=5000,
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0],
        callbacks=[
            lgb.early_stopping(int(50 + 0.05 / learning_rate)),
            lgb.log_evaluation(period=50)
        ]
    )

    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5



Dejo comentadas las celdas a continuación para mostrar la función de ganancia y rango de hiperparámetros utilizados para la optimización.

In [ ]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm_1010_v3.db"
study_name = "1010_v3_lgbm"

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

In [ ]:
'''study.optimize(objective, n_trials=100)''' #No llegó a los 100. Lo corté al trial 88.

In [ ]:
'''print("Mejor valor objetivo:", study.best_value)
print("Mejores hiperparámetros:", study.best_params)

# DataFrame con todos los trials
df = study.trials_dataframe(attrs=("number", "value", "params"))
df'''


In [ ]:
'''best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model: {best_iter}")


best_params = study.best_trial.params

params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'verbose': 0,
    'seed': semillas[0],

    'num_leaves': best_params['num_leaves'],
    'learning_rate': best_params['learning_rate'],
    'min_data_in_leaf': best_params['min_data_in_leaf'],
    'feature_fraction': best_params['feature_fraction'],
    'bagging_fraction': best_params['bagging_fraction'],
    
    'reg_alpha': best_params['reg_alpha'],
    'reg_lambda': best_params['reg_lambda'],
    'max_depth': best_params['max_depth'] 
}

train_data = lgb.Dataset(X_train,
                         label=y_train_binaria2,
                         weight=w_train)

model = lgb.train(params,
                  train_data,
                  num_boost_round=best_iter)'''

In [ ]:
'''importances = model.feature_importance()
feature_names = X_train.columns.tolist()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values('importance', ascending=False)
importance_df[importance_df['importance'] > 0].head(20)'''


In [ ]:
'''model.save_model(modelos_path + 'lgb_1010_v3_1.txt')'''

## Test

Levanto el modelo.

In [ ]:
model = lgb.Booster(model_file=modelos_path + 'lgb_1010_v3_1.txt')

Predigo abril.

In [ ]:
y_pred_lgm = model.predict(X_test)

In [ ]:
ganancia = np.where(y_test_binaria1 == 1, ganancia_acierto, 0) - np.where(y_test_binaria1 == 0, costo_estimulo, 0)

idx = np.argsort(y_pred_lgm)[::-1]

ganancia = ganancia[idx]
y_pred_lgm = y_pred_lgm[idx]

ganancia_cum = np.cumsum(ganancia)


El máximo de ganancia se alcanza en x = 0.036170 con ganancia = 367480000.00 y corresponde a 10465 envíos.

In [ ]:
piso_envios = 4000
techo_envios = 20000

x_vals = y_pred_lgm[piso_envios:techo_envios]
y_vals = ganancia_cum[piso_envios:techo_envios]

idx_max = np.argmax(y_vals)
x_max = x_vals[idx_max]
y_max = y_vals[idx_max]

plt.figure(figsize=(10, 6))
plt.plot(x_vals, y_vals, label='Ganancia LGBM')
plt.axvline(x=0.025, color='g', linestyle='--', label='Punto de corte a 0.025')
plt.axvline(x=x_max, color='r', linestyle='--', label=f'Máximo (x={x_max:.4f})')
plt.scatter(x_max, y_max, color='r') 
plt.title('Curva de Ganancia')
plt.xlabel('Predicción de probabilidad')
plt.ylabel('Ganancia')
plt.legend()
plt.show()

print(f"El máximo de ganancia se alcanza en x = {x_max:.6f} con ganancia = {y_max:.2f}")


In [ ]:
piso_envios = 4000
techo_envios = 20000

ganancia_max = ganancia_cum.max()
gan_max_idx = np.where(ganancia_cum == ganancia_max)[0][0]

plt.figure(figsize=(10, 6))
plt.plot(range(piso_envios, len(ganancia_cum[piso_envios:techo_envios]) + piso_envios), ganancia_cum[piso_envios:techo_envios], label='Ganancia LGBM')
plt.axvline(x=gan_max_idx, color='g', linestyle='--', label=f'Punto de corte a la ganancia máxima {gan_max_idx}')
plt.axhline(y=ganancia_max, color='r', linestyle='--', label=f'Ganancia máxima {ganancia_max}')
plt.title('Curva de Ganancia')
plt.xlabel('Clientes')
plt.ylabel('Ganancia')
plt.legend()
plt.show()


## Entrenamiento final y predicción de Junio

Entreno con los continúa de marzo más los baja+1 y baja+2 de enero, febrero, marzo y abril.

In [ ]:
dfs = [df_enero, df_febrero, df_marzo, df_abril, df_mayo, df_junio]

clientes_baja = set()
for dfx in dfs:
    clientes_baja.update(
        dfx.loc[dfx['clase_ternaria'].isin(['baja+1', 'baja+2']), 'numero_de_cliente'].to_numpy()
    )
gc.collect()

train_data_parts = []
meses_interes = {202101, 202102, 202103, 202104} 

for dfx in dfs:
    mask = (
        ~((dfx['numero_de_cliente'].isin(clientes_baja)) &
          (dfx['clase_ternaria'] == 'continua')) &
        (dfx['foto_mes'].isin(meses_interes))
    )
    train_data_parts.append(dfx.loc[mask])
    gc.collect()

_train_data = pd.concat(train_data_parts, ignore_index=True)
train_data = _train_data[~(_train_data['foto_mes'].isin([202101, 202102, 202104]) & (_train_data['clase_ternaria'] == 'continua'))]

del dfs, dfx, clientes_baja, train_data_parts, mask, _train_data
gc.collect()

Para la predicción, entreno 5 modelos (mismos hiperparámetros, 5 semillas distintas) y los promedio.

In [ ]:
modelos = [0,0,0,0,0]
y_pred_futuro = [0,0,0,0,0]

X_kaggle = df_junio.copy()
X_kaggle= X_kaggle.drop(['clase_ternaria', 'clase_peso','clase_binaria1','clase_binaria2','clase_resumida','foto_mes'], axis=1)
X_futuro = X_kaggle.copy()

best_iter = study.best_trial.user_attrs["best_iter"]
best_params = study.best_trial.params

for i in range(len(semillas)):
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'verbose': 0,
        'seed': semillas[i],

        'num_leaves': best_params['num_leaves'],
        'learning_rate': best_params['learning_rate'],
        'min_data_in_leaf': best_params['min_data_in_leaf'],
        'feature_fraction': best_params['feature_fraction'],
        'bagging_fraction': best_params['bagging_fraction'],
        
        'reg_alpha': best_params['reg_alpha'],
        'reg_lambda': best_params['reg_lambda'],
        'max_depth': best_params['max_depth'] 
    }

    train_data = lgb.Dataset(X_train,
                            label=y_train_binaria2,
                            weight=w_train)

    modelos[i] = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)

    y_pred_futuro[i] = modelos[i].predict(X_futuro)

In [ ]:
pred_matrix = np.column_stack(y_pred_futuro)
y_pred_promedio = pred_matrix.mean(axis=1) 

Para el umbral elijo 0.04, un poco más del óptimo para Abril por razones mágicas.

In [ ]:
UMBRAL_PROBABILIDAD = 0.04

df_submission = pd.DataFrame({
    'numero_de_cliente': X_futuro['numero_de_cliente'].values,
    'probability': y_pred_promedio
})

df_submission['Predicted'] = (df_submission['probability'] >= UMBRAL_PROBABILIDAD).astype(int)

df_kaggle_final = df_submission[['numero_de_cliente', 'Predicted']]

nombre_archivo = '1010_v3_1_4.csv'
df_kaggle_final.to_csv(nombre_archivo, index=False)